<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/training_smell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,745 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download.nvidia.co

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# from pyspark.sql import SparkConf, SparkContext

In [ ]:
# Start a SparkSession
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [ ]:
MAX_MEMORY = "5g"

spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [ ]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)

# Show DataFrame
# df.show()

In [ ]:
# Check the schema for numeric conversion
# df.printSchema

In [ ]:
# df.count()

In [ ]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')

In [ ]:
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')

In [ ]:
# confirm look, smell, taste, overall, and score
# df.show()

In [ ]:
# df.count()

In [ ]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))

In [ ]:
df = df.withColumn('smell_numeric', df['smell'].cast('float'))

In [ ]:
df = df.withColumn('taste_numeric', df['taste'].cast('float'))

In [ ]:
df = df.withColumn('feel_numeric', df['feel'].cast('float'))

In [ ]:
df = df.withColumn('overall_numeric', df['overall'].cast('float'))

In [ ]:
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [ ]:
# Confirm data types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string, look_numeric: float, smell_numeric: float, taste_numeric: float, feel_numeric: float, overall_numeric: float, score_numeric: float]>

In [ ]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [ ]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [ ]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
# beer_df.show()

In [ ]:
# beer_df.count()

In [ ]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
# style_df.show()

In [ ]:
# style_df.count()

In [ ]:
# clean the text by removing punctuation
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-

In [ ]:
# text_df.count()

In [ ]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# combined_df.count()

In [ ]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)
# final_df.show()

In [ ]:
# final_df.show()

In [ ]:
# final_df.count()

In [ ]:
final_df = final_df.select('smell_numeric','features')

In [ ]:
final_df.show()

+-------------+--------------------+
|smell_numeric|            features|
+-------------+--------------------+
|          4.0|[motorbreathimper...|
|          4.0|[paybackpilsner, ...|
|          4.0|[paybackpilsner, ...|
|         3.25|[paybackpilsner, ...|
|         3.75|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|         3.25|[paybackpilsner, ...|
|         3.75|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|          3.0|[hellhammer, ryeb...|
|          4.0|[hellhammer, ryeb...|
|          3.0|[vanillaporter, a...|
|          4.0|[vanillaporter, a...|
|          3.0|[vanillaporter, a...|
|         3.75|[vanillaporter, a...|
|          2.5|[vanillaporter, a...|
|          4.0|[vanillaporter, a...|
|         4.25|[vanillaporter, a...|
|         3.75|[vanillaporter, a...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
# Create w2v model hyperparameters
# word2vec = Word2Vec(
#    vectorSize=100,
#    seed=42,
#    inputCol="features",
#    outputCol="model"
# ).setMaxIter(2)

In [ ]:
# Fit the model, this takes time
# w2v_review_model = word2vec.fit(final_df)

In [ ]:
# show some model vectors
# w2v_review_model.getVectors().show()

In [ ]:
# check the model accuracy by checking synonyms 
#w2v_review_model.findSynonymsArray("light", 5)

In [ ]:
# write the model to disk
#w2v_review_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/review.md")

In [ ]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [ ]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('medium', 0.6914238929748535),
 ('mostly', 0.6894241571426392),
 ('ligh', 0.6799790859222412),
 ('slight', 0.6678677201271057),
 ('mild', 0.6617092490196228)]

In [ ]:
# Transform the mode
w2v_model_df = w2v_loaded_model.transform(final_df)

In [ ]:
w2v_model_df.show()

+-------------+--------------------+--------------------+
|smell_numeric|            features|               model|
+-------------+--------------------+--------------------+
|          4.0|[motorbreathimper...|[0.01448507781083...|
|          4.0|[paybackpilsner, ...|[-0.0253503741696...|
|          4.0|[paybackpilsner, ...|[0.18410651786969...|
|         3.25|[paybackpilsner, ...|[0.06293463234656...|
|         3.75|[paybackpilsner, ...|[0.05097741415932...|
|          3.5|[paybackpilsner, ...|[-0.0508729857910...|
|         3.25|[paybackpilsner, ...|[0.00700588056351...|
|         3.75|[paybackpilsner, ...|[0.05808807828245...|
|          3.5|[paybackpilsner, ...|[0.00684954653171...|
|          3.5|[paybackpilsner, ...|[0.10990913212299...|
|          3.0|[hellhammer, ryeb...|[0.03865336287325...|
|          4.0|[hellhammer, ryeb...|[-0.0377690620720...|
|          3.0|[vanillaporter, a...|[0.07743978367320...|
|          4.0|[vanillaporter, a...|[0.03384895505777...|
|          3.0

In [ ]:
w2v_model_df = w2v_model_df.select('smell_numeric', 'model')

In [ ]:
# Prepare the hyperparameters
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
lin_model = LinearRegression(
   maxIter=5,
   regParam=0.3,
   featuresCol="model",
   labelCol="smell_numeric"
)

In [ ]:
w2v_model_df.cache()
w2v_model_df.show()

+-------------+--------------------+
|smell_numeric|               model|
+-------------+--------------------+
|          4.0|[0.01448507781083...|
|          4.0|[-0.0253503741696...|
|          4.0|[0.18410651786969...|
|         3.25|[0.06293463234656...|
|         3.75|[0.05097741415932...|
|          3.5|[-0.0508729857910...|
|         3.25|[0.00700588056351...|
|         3.75|[0.05808807828245...|
|          3.5|[0.00684954653171...|
|          3.5|[0.10990913212299...|
|          3.0|[0.03865336287325...|
|          4.0|[-0.0377690620720...|
|          3.0|[0.07743978367320...|
|          4.0|[0.03384895505777...|
|          3.0|[0.02083584428247...|
|         3.75|[0.13457064807415...|
|          2.5|[0.05896031627939...|
|          4.0|[-0.0022915382129...|
|         4.25|[0.10159241169859...|
|         3.75|[0.08570952849079...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
# split training and testing for the model (for temporary make the training set small as processing takes time)
training, testing = w2v_model_df.randomSplit([0.80, 0.20])

In [ ]:
training.show()

+-------------+--------------------+
|smell_numeric|               model|
+-------------+--------------------+
|          1.0|[-0.2247871309518...|
|          1.0|[-0.1815563549420...|
|          1.0|[-0.1415252586205...|
|          1.0|[-0.1372455065138...|
|          1.0|[-0.1294552323718...|
|          1.0|[-0.1141534168273...|
|          1.0|[-0.1036582394236...|
|          1.0|[-0.0947362417562...|
|          1.0|[-0.0847558726867...|
|          1.0|[-0.0847325982504...|
|          1.0|[-0.0745497764925...|
|          1.0|[-0.0710590472118...|
|          1.0|[-0.0695140659809...|
|          1.0|[-0.0688783476638...|
|          1.0|[-0.0688170208886...|
|          1.0|[-0.0672105080183...|
|          1.0|[-0.0629410125677...|
|          1.0|[-0.0628406693576...|
|          1.0|[-0.0597076173005...|
|          1.0|[-0.0594514317768...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
training_pandas_df = training.toPandas()

In [ ]:
training_pandas_df.head()

,smell_numeric,model
0,1.0,"[-0.2247871309518814, 0.008813098073005676, -0..."
1,1.0,"[-0.18155635494206632, 0.10784403341157095, -0..."
2,1.0,"[-0.14152525862058002, -0.20047504703203836, -..."
3,1.0,"[-0.1372455065138638, 0.2034167988342233, -0.1..."
4,1.0,"[-0.12945523237188655, -0.1655172910541296, 0...."


In [ ]:
training_pandas_df.dtypes

smell_numeric    float32
model             object
dtype: object

In [ ]:
from sklearn.linear_model import LinearRegression
pandas_lin_model = LinearRegression()

In [ ]:
lin_smell_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['smell_numeric'])

In [ ]:
lin_smell_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['smell_numeric'])

0.3428030525335479

In [ ]:
lin_smell_train_model.coef_

array([-4.78942088e-02, -5.20566909e-01, -3.07614275e-01, -1.19515586e+00,
        4.50947061e-01, -2.74989444e-01,  6.44216787e-01, -7.06570779e-02,
       -5.57731360e-01, -4.28823154e-01, -1.17645343e+00,  3.82724238e-01,
        1.74904202e-01,  2.15783032e-01,  4.16616951e-01,  4.39857849e-01,
       -6.99680104e-01, -1.89456981e-01, -1.07471016e+00, -3.88463107e-01,
        9.21588472e-01, -8.09219411e-01, -6.96887984e-02,  8.67884992e-01,
        6.94793083e-01,  1.19871038e-01,  1.22082160e-01, -1.91836405e+00,
       -1.18287443e-01,  1.49319140e-01,  1.04529020e-01, -1.02235387e+00,
        8.69644892e-01, -4.91629782e-01, -7.20361693e-01, -1.44621529e+00,
        1.68267955e-01, -1.99062671e-01, -6.44779972e-01, -3.55584669e-01,
        4.18828824e-01,  9.60607080e-02, -3.10187174e-01, -3.00793892e-01,
       -4.59322933e-01, -7.62742772e-01, -8.61649362e-02,  1.44696272e-01,
       -4.39899568e-01,  9.67581870e-02,  3.71368755e-02, -2.07851878e-01,
        1.26474502e+00, -

In [ ]:
# Fit the look review to a model (takes time)
#lin_look_train_model = lin_model.fit(training)

In [ ]:
# Save the model for later use
#lin_look_train_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/lin_look_train.md")

In [ ]:
# Read the model
loaded_lin_look_train_model = LinearRegressionModel.load("/content/gdrive/MyDrive/Project3_BeerQuality/lin_look_train.md")

In [ ]:
# Transform the training data to peek at the predictions
lin_look_df = loaded_lin_look_train_model.transform(training)

In [ ]:
# show a sample of the predictions
lin_look_df.select('look_numeric', 'prediction', 'Final_text').show(truncate=False)

+------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Experiment
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#evaluator = MulticlassClassificationEvaluator(
#    labelCol="look_numeric", 
#    predictionCol="prediction", 
#    metricName="accuracy"
#)

#accuracy = evaluator.evaluate(lin_look_df)

#print("Accuracy of model at predicting reviews was: %f" % accuracy)

Accuracy of model at predicting reviews was: 0.000000


In [ ]:
print(training.rdd.getNumPartitions())

14


In [ ]:
partitioned_df = training.repartition(2)

In [ ]:
print(partitioned_df.rdd.getNumPartitions())

2


In [ ]:
print(partitioned_df.count())

19959


In [ ]:
print(spark.sparkContext.defaultParallelism)

2


In [ ]:
s = spark.sparkContext._jsc.sc().getExecutorMemoryStatus().keys()
print(s)
l = str(s).replace("Set(","").replace(")","").split(", ")
print(l)

d = set()
for i in l:
    d.add(i.split(":")[0])
print(len(d))

print(spark.sparkContext._jsc.sc().getExecutorMemoryStatus().size())

Set(492190d9606c:38029)
['492190d9606c:38029']
1
1


In [ ]:
spark.sparkContext._conf.getAll() 

[('spark.driver.port', '33275'),
 ('spark.app.id', 'local-1615332036948'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '492190d9606c'),
 ('spark.app.name', 'word2vec'),
 ('spark.ui.showConsoleProgress', 'true')]

In [ ]:
spark.sparkContext._conf.get('spark.driver.maxResultSize')

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 49
model name	: AMD EPYC 7B12
stepping	: 0
microcode	: 0x1000065
cpu MHz		: 2250.000
cache size	: 512 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
bugs		: sysret_ss_attrs spectre_v1 spectre_v2 spec_store_bypass
bogomips	: 4500.00
TLB size	: 307

In [ ]:
!cat /proc/meminfo

MemTotal:       13333568 kB
MemFree:          373764 kB
MemAvailable:   11164900 kB
Buffers:          115264 kB
Cached:         10581148 kB
SwapCached:            0 kB
Active:          4418788 kB
Inactive:        8105484 kB
Active(anon):    1701420 kB
Inactive(anon):      360 kB
Active(file):    2717368 kB
Inactive(file):  8105124 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               968 kB
Writeback:             0 kB
AnonPages:       1827760 kB
Mapped:           140948 kB
Shmem:              1124 kB
Slab:             336356 kB
SReclaimable:     294532 kB
SUnreclaim:        41824 kB
KernelStack:        5716 kB
PageTables:         9956 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666784 kB
Committed_AS:    5137308 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1024 kB
AnonHugePages:   

In [ ]:
!ps -ef | grep java

root        1232      62 56 18:31 ?        00:00:50 /usr/lib/jvm/java-8-openjdk-amd64/bin/java -cp /content/spark-2.4.7-bin-hadoop2.7/conf/:/content/spark-2.4.7-bin-hadoop2.7/jars/* -Xmx5g org.apache.spark.deploy.SparkSubmit --conf spark.executor.memory=5g --conf spark.driver.memory=5g --conf spark.app.name=word2vec pyspark-shell
root        1438      62  0 18:32 ?        00:00:00 /bin/bash -c ps -ef | grep java
root        1440    1438  0 18:32 ?        00:00:00 grep java


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Mar 11 17:36:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!
